In [40]:
import pandas as pd
import numpy as np

In [41]:
df = pd.read_csv('vgames2.csv', index_col='Unnamed: 0')

# df

In [42]:
# 판매 단위를 동기화한다.
def preProcess(df: pd.DataFrame, key: str):
    sub = df[key]

    type(sub)
    k = sub.str.match(r'([.|0-9]*[kK])')
    m = sub.str.match(r'([0-9]*.[0-9]*[mM])')

    df.loc[k, key] = df.loc[k, key].str[:-1]
    df.loc[m, key] = df.loc[m, key].str[:-1].astype(np.float64) * 1000

    # df.loc[k, key]

# match = df['NA_Sales'].str.match(r'([.|0-9]*[kK])')

preProcess(df, 'NA_Sales')
preProcess(df, 'EU_Sales')
preProcess(df, 'JP_Sales')
preProcess(df, 'Other_Sales')

# df[match]

In [43]:
# 결측치 제거 1 : 단순 제거
# 결측치 제거 2안 : 판매량이 X 이상인 경우 제외하고 결측치 제거
# 결측치 제거 3안 : 일부 결측치는 다른 걸로 대체하고 그외 제거
# from operator import index
print(df.isnull().sum())

nYear = df['Year'].isna()
nGenre = df['Genre'].isna()
nPublisher = df['Publisher'].isna()

print(nYear.sum())
print(nGenre.sum())
print(nPublisher.sum())

# df.loc[nYear.index.tolist()]    # 16598 rows × 9 columns
# df.loc[nYear]   # 271 rows × 9 columns

# df.loc[nGenre.index.tolist()] # 16598 rows × 9 columns
# df.loc[nGenre] # 50 rows × 9 columns

# df.loc[nPublisher.index.tolist()] # 16598 rows × 9 columns
# df.loc[nPublisher] # 58 rows × 9 columns

df.drop(df[nYear].index, inplace=True)
df.drop(df[nGenre].index, inplace=True)
df.drop(df[nPublisher].index, inplace=True)

# df
print(df.isnull().sum())


Name             0
Platform         0
Year           271
Genre           50
Publisher       58
NA_Sales         0
EU_Sales         0
JP_Sales         0
Other_Sales      0
dtype: int64
271
50
58
Name           0
Platform       0
Year           0
Genre          0
Publisher      0
NA_Sales       0
EU_Sales       0
JP_Sales       0
Other_Sales    0
dtype: int64


C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\1219568242.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop(df[nGenre].index, inplace=True)
C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\1219568242.py:26: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop(df[nPublisher].index, inplace=True)


In [44]:
# 중복 제거
df.duplicated().sum()

0

In [45]:
# 데이터타입 정리
df.info(); print()

df['Year'] = df['Year'].astype(np.int32).astype(object)

df['NA_Sales'] = df['NA_Sales'].astype(np.float64)
df['EU_Sales'] = df['EU_Sales'].astype(np.float64)
df['JP_Sales'] = df['JP_Sales'].astype(np.float64)
df['Other_Sales'] = df['Other_Sales'].astype(np.float64)

df.info()
# df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  float64
 3   Genre        16241 non-null  object 
 4   Publisher    16241 non-null  object 
 5   NA_Sales     16241 non-null  object 
 6   EU_Sales     16241 non-null  object 
 7   JP_Sales     16241 non-null  object 
 8   Other_Sales  16241 non-null  object 
dtypes: float64(1), object(8)
memory usage: 1.2+ MB

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16241 entries, 1 to 16598
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         16241 non-null  object 
 1   Platform     16241 non-null  object 
 2   Year         16241 non-null  object 
 3   Genre        16241 non-null  object 
 4   Publisher   

In [46]:
# 이상치 연도 정리
# 1. 0 ~ 1980 사이 데이터 검출
# 2. 총 판매량 1k 기준으로 삭제여부 판단
    # 2-1. 1k 넘는 경우
        # 삭제 보류
    # 2-2. 1k 안 넘는 경우
        # 삭제
con1 = df['Year'] < 1980
con2 = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'] < 1
con3 = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'] >= 1

# 1980 이하 연도의 오류값을 가진 행을 분류한다.
(df[con1])[con2]    # 1980 이하 연도에서 판매량 1k 미만인가?
(df[con1])[con3]    # 1980 이하 연도에서 판매량 1k 이상인가?

# print(len(df[con1].index))  # 이상치 100개
# print(len(df[con1][con2].index)) # 이상치 1k 미만 분류 87개
# print(len(df[con1][con3].index)) # 이상치 1k 이상 분류 13개

df.drop((df[con1])[con2].index, inplace=True)

# print(len(df[con1].index))  # 이상치 13개 (1k 미만 데이터를 지웠으므로 13개만 남음)

df[con1]

C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\1279893734.py:13: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  (df[con1])[con2]    # 1980 이하 연도에서 판매량 1k 미만인가?
C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\1279893734.py:14: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  (df[con1])[con3]    # 1980 이하 연도에서 판매량 1k 이상인가?
C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\1279893734.py:20: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df.drop((df[con1])[con2].index, inplace=True)
C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\1279893734.py:24: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[con1]


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
769,NASCAR Thunder 2004,PS2,3,Racing,Electronic Arts,0.53,0.41,0.00,0.14
1221,Ratchet & Clank: Going Commando,PS2,3,Platform,Sony Computer Entertainment,1.44,0.83,0.26,0.25
1365,LEGO Star Wars II: The Original Trilogy,PS2,6,Action,LucasArts,1.85,0.64,0.01,0.19
2455,SOCOM: U.S. Navy SEALs Fireteam Bravo 2,PSP,6,Shooter,Sony Computer Entertainment,0.81,0.11,0.00,0.13
4011,Uncharted 4: A Thief's End,PS4,16,Shooter,Sony Computer Entertainment,1300.00,2.07,0.18,0.65
4366,Adventure Island,NES,86,Platform,Hudson Soft,0.00,0.00,1.05,0.00
4728,NBA 2K15,PS4,14,Sports,Take-Two Interactive,1.46,0.54,0.01,0.37
6114,Kingdom Hearts: Birth by Sleep,PSP,10,Role-Playing,Square Enix,0.63,0.42,0.75,0.26
6160,Brink,X360,11,Shooter,Bethesda Softworks,0.59,0.35,0.01,0.09
8682,NCAA Football 2002,PS2,1,Sports,Electronic Arts,0.77,0.60,0.00,0.20


In [47]:
# 이상치 연도 정리 2
con1 = df['Year'] < 1980
con2 = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'] < 1
con3 = df['NA_Sales'] + df['EU_Sales'] + df['JP_Sales'] + df['Other_Sales'] >= 1

dTemp = df.copy()

# # 데이터프레임 상태로 가져온다.
df1 = dTemp[con1][con3]

df1['Year'].astype(str) # 잠시 문자열로 변경

con4 = df1['Year'] < 50
con5 = df1['Year'] >= 50

df2 = df1[con4]
df3 = df1[con5]

# df2['Year'] = '20' + df2['Year']

# df2['Year'].str.cat('20' + df2['Year'])
df2['Year'] = 2000 + df2['Year']
df1[con4]['Year'] = df2['Year']

df1.loc[df2.index] = df2
df1.loc[df2.index]

df1.loc[4366, 'Year'] = 1986

df1.shape

df.loc[df1.index] = df1
# df.loc[df1.index]

df[con1]

C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\2097228371.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df1 = dTemp[con1][con3]
C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\2097228371.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Year'] = 2000 + df2['Year']
C:\Users\rimmo\AppData\Local\Temp\ipykernel_13640\2097228371.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1[con4]['Year'] = df2['Year']


,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales
769,NASCAR Thunder 2004,PS2,2003,Racing,Electronic Arts,0.53,0.41,0.00,0.14
1221,Ratchet & Clank: Going Commando,PS2,2003,Platform,Sony Computer Entertainment,1.44,0.83,0.26,0.25
1365,LEGO Star Wars II: The Original Trilogy,PS2,2006,Action,LucasArts,1.85,0.64,0.01,0.19
2455,SOCOM: U.S. Navy SEALs Fireteam Bravo 2,PSP,2006,Shooter,Sony Computer Entertainment,0.81,0.11,0.00,0.13
4011,Uncharted 4: A Thief's End,PS4,2016,Shooter,Sony Computer Entertainment,1300.00,2.07,0.18,0.65
4366,Adventure Island,NES,1986,Platform,Hudson Soft,0.00,0.00,1.05,0.00
4728,NBA 2K15,PS4,2014,Sports,Take-Two Interactive,1.46,0.54,0.01,0.37
6114,Kingdom Hearts: Birth by Sleep,PSP,2010,Role-Playing,Square Enix,0.63,0.42,0.75,0.26
6160,Brink,X360,2011,Shooter,Bethesda Softworks,0.59,0.35,0.01,0.09
8682,NCAA Football 2002,PS2,2001,Sports,Electronic Arts,0.77,0.60,0.00,0.20


In [48]:
# 추가 항목 부여

df['Total_Sales'] = df["NA_Sales"] + df["EU_Sales"] + df['JP_Sales'] + df['Other_Sales']
df

,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Total_Sales
1,Candace Kane's Candy Factory,DS,2008,Action,Destineer,0.04,0.00,0.00,0.00,0.04
2,The Munchables,Wii,2009,Action,Namco Bandai Games,0.17,0.00,0.00,0.01,0.18
3,Otome wa Oanesama Boku ni Koi Shiteru Portable,PSP,2010,Adventure,Alchemist,0.00,0.00,0.02,0.00,0.02
4,Deal or No Deal: Special Edition,DS,2010,Misc,Zoo Games,0.04,0.00,0.00,0.00,0.04
5,Ben 10 Ultimate Alien: Cosmic Destruction,PS3,2010,Platform,D3Publisher,0.12,0.09,0.00,0.04,0.25
...,...,...,...,...,...,...,...,...,...,...
16594,Ice Age 2: The Meltdown,GC,2006,Platform,Vivendi Games,0.15,0.04,0.00,0.01,0.20
16595,Rainbow Islands: Revolution,PSP,2005,Action,Rising Star Games,0.01,0.00,0.00,0.00,0.01
16596,NBA 2K16,PS3,2015,Sports,Take-Two Interactive,0.44,0.19,0.03,0.13,0.79
16597,Toukiden: The Age of Demons,PSV,2013,Action,Tecmo Koei,0.05,0.05,0.25,0.03,0.38


In [282]:
# 단순 정리된 데이터 csv화
df.to_csv('vgames2_clean2.csv')